# <font color=#0404B4>**SEMINAR - PHÂN TÍCH DỮ LIỆU THÔNG MINH** </font>

### <font color=#0404B4> Bảng thành viên nhóm </font>
|<center><div style="width:150px">MSSV</div><center>|<center><div style="width:290px">Họ và tên</div><center>|
|---------- |:-------------:|
| <center>20120032<center>  | <center>Phan Trường An<center> |
| <center>20120037<center>  | <center>Trần Thị Minh Anh<center> |
| <center>20120041<center>  | <center>Trần Kim Bảo<center> |
| <center>20120053<center>  |   <center>Nguyễn Thành Đạt<center>   | 
| <center>20120071<center>  |<center>Nguyễn Thị Bích Hà<center> |
| <center>20120095<center>  | <center>Ngô Quang Hưng<center> |

# <font color=#0404B4>Import thư viện</font>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

<a name="2"></a>
# <font color=#0404B4> A. Tiền xử lý dữ liệu</font>


### 1. Đọc dữ liệu

In [ ]:
item_category_df=pd.read_csv("item_categories.csv")
item_category_df.shape

In [ ]:
items_df=pd.read_csv("items.csv")
items_df.shape

In [ ]:
sales_train_df=pd.read_csv("sales_train.csv")
sales_train_df.shape

In [ ]:
sample_submission_df=pd.read_csv("sample_submission.csv")
sample_submission_df.shape

In [ ]:
shops_df=pd.read_csv("shops.csv")
shops_df.shape

In [ ]:
test_df=pd.read_csv("test.csv")
test_df.shape

### 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- `item_category_df`: Mỗi dòng là thông tin về danh mục sản phẩm gồm: tên danh mục sản phẩm và id của danh mục sản phẩm đó.
- `items_df`: Mỗi dòng là thông tin về sản phẩm gồm: tên sản phẩm, id của sản phẩm và id của danh mục chứa sản phẩm đó.
- `sales_train_df`: Mỗi dòng là thông tin về bán hàng gồm: ngày bán hàng, thuộc kì bán nào(1 kì ở đây tương ứng với 1 tháng), id của cửa hàng bán sản phẩm đó, id của sản phẩm bán, thông tin về giá của sản phẩm và số lượng bán sản phẩm đó trong 1 ngày.
- `sample_submission_df`: Hướng dẫn định dạng nộp bài gồm các thông tin: ID(thứ tự) dự đoán, số lượng sản phẩm bán ra trong tháng.
- `shops_df`: Mỗi dòng là thông tin về cửa hàng gồm: tên cửa hàng và id của cửa hàng đó
- `test_df`: Chứa dữ liệu kiểm tra cho mô hình dự đoán gồm ID dự đoán, id cửa hàng và id sản phẩm.

### 3. Có dữ liệu bị trùng lặp ở các dataframe không

In [ ]:
def check_duplicated(dataframe):
    duplicated_rows = dataframe.duplicated()

    # In các hàng bị lặp lại
    if len(dataframe[duplicated_rows])>0:
        print('Có dòng dữ liệu bị lặp')
        display(dataframe[duplicated_rows])
    else: 
        print('Không có dòng dữ liệu nào bị lặp')


Dataframe `item_category_df`

In [ ]:
check_duplicated(item_category_df)

Dataframe `items_df`

In [ ]:
check_duplicated(items_df)

Dataframe `sales_train_df`

In [ ]:
check_duplicated(sales_train_df)

Xóa luôn các mẫu dữ liệu bị trùng

In [ ]:
sales_train_df.drop_duplicates(inplace=True)

Dataframe `sample_submission_df`

In [ ]:
check_duplicated(sample_submission_df)

Dataframe `shops_df`

In [ ]:
check_duplicated(shops_df)

Dataframe `test_df`

In [ ]:
check_duplicated(test_df)

<h3> 4. Mỗi cột có ý nghĩa gì?</h3>

|STT|Tên cột dữ liệu | <center>Mô tả<center> | Đơn vị |
|--|:------:| ---| --- |
|1|item_category_name|<center>Tên danh mục sản phẩm<center>|
|2|item_category_id| <center>Mã định danh của danh mục sản phẩm<center>|
|3|item_name|<center>Tên sản phẩm<center>| 
|4|item_id| <center>Mã định danh sản phẩm<center>||
|5|date|<center>Ngày bán hàng<center>|| 
|6|date_block_num|<center>Thứ tự kỳ bán hàng *(bắt đầu từ 0)*<center>|
|7|shop_id|<center>Mã định danh cửa hàng<center>| |
|8|item_price|<center>Giá sản phẩm<center>|<center>Rúp<center>|
|9|item_cnt_day|<center>Số lượng bán ra/ số lượng mất trong `1 ngày`<center>|<center>Sản phẩm <center>|
|10|ID|<center>Mã định danh<center>| |
|11|item_cnt_month|<center>Số lượng bán ra/ số lượng mất trong `1 tháng`<center>|<center>Sản phẩm <center>|
|12|shop_name|<center>Tên cửa hàng<center>| |


### 5. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp?



Dataframe `item_category_df`

In [ ]:
item_category_df.columns

In [ ]:
item_category_df.info()

Dataframe `items_df`

In [ ]:
items_df.columns

In [ ]:
items_df.info()

Dataframe `sales_train_df`

In [ ]:
sales_train_df.columns

In [ ]:
sales_train_df.info()

Dataframe `sample_submission_df`

In [ ]:
sample_submission_df.columns

In [ ]:
sample_submission_df.info()

Dataframe `shops_df`

In [ ]:
shops_df.columns

In [ ]:
shops_df.info()

Dataframe `test_df`

In [ ]:
test_df.columns

In [ ]:
test_df.info()


##### Cột `date` của dataframe `sales_train_df` đang có kiểu dữ liệu `object`. Thực hiện xử lý đưa về đúng kiểu dữ liệu `datime64[ns]`.

In [ ]:
sales_train_df.date=pd.to_datetime(sales_train_df.date,format="%d.%m.%Y")

In [ ]:
sales_train_df.info()


### 7. Với mỗi cột, các giá trị được phân bố như thế nào?

- Nhận xét:
    - Các dataframe như: `item_category_df`, `items_df`, `sample_submission_df`, `shops_df` và `test_df`, là những dataframe chứa thông tin chi tiết đối đối với dataframe `sales_train_df`. Nên sẽ chỉ tập trung xem phân bố dữ liệu của dataframe `sales_train_df`
    - Với các dữ liệu trong dataframe `sales_train_df` chỉ có dạng số.

#### Cột có kiểu dữ liệu dạng số, và phân bố:


In [ ]:
numerical_cols = sales_train_df[sales_train_df.select_dtypes(exclude=['object']).columns]

def missing_ratio(c):
    return  c.isna().sum()/len(c) * 100

def min(c):
    return c.min()

def max(c):
    return c.max()

nume_col_profiles_df = numerical_cols.agg([missing_ratio, min, max])
nume_col_profiles_df

- Nhận xét:
    - `date`: dữ liệu bán hàng được thu thập từ ngày 01-01-2013 đến ngày 31-10-2015.
    - `date_block_num`: Chia dữ liệu thành 34 kì (1 kì=1 tháng)
    - `shop_id`: ID cửa hàng có giá trị nhỏ nhất là 0, lớn nhất là 59.
    - `item_id`: ID sản phẩm có giá trị nhỏ nhất là 0, lớn nhất là 22169.
    - `item_price`: Sản phẩm có giá tiền nhỏ nhất -1, lớn nhất là 307980 (Rúp).
    - `item_cnt_day`: Số lượng sản phẩm đó được bán ra. Số lượng sản phẩm bị mất lớn nhất là 22 sản phẩm, có thể nhiều lý do như hư hỏng, thất lạc trong quá trình vận chuyển,... Số lượng sản phẩm được bán ra lớn nhất là 2169 sản phẩm. 

###  8. Có cột nào cần phải tiền xử lý hay không?


Kiểm tra thử số lượng `item_price` có giá bị âm

In [ ]:
# sales_train_df[sales_train_df.item_price<0]
sales_train_df[sales_train_df.item_cnt_day<0]

Với chỉ 1 giá sản phẩm bị âm nên ta có thể thực hiện loại xóa dữ liệu đó.

In [ ]:
index_drop=sales_train_df[sales_train_df.item_price<0].index
sales_train_df.drop(index_drop, inplace=True)

<a name="2"></a>
# <font color=#0404B4> B. Khám phá dữ liệu</font>


### Tìm ra tháng có tiền từ doanh số bán hàng cao nhất. Phân tích tương quan giữa số tiền mỗi shop kiếm được với số mặt hàng các shop đã bán.

#### Bước 1: Tiến hành gom nhóm và tính tổng số tiền bán được của tất cả các shop.

In [ ]:
df_income=sales_train_df.copy()
df_income['month']=df_income['date'].dt.month
df_income['year']=df_income['date'].dt.year.astype(str)
df_income['income']=df_income['item_cnt_day']*df_income['item_price']
df_income=df_income.groupby(['year','month']).sum()
df_income=df_income.reset_index()[['year','month','income']]
df_income.head()

#### Bước 2: Trực quan bằng `line chart`.

- Dùng `line chart` có thể giúp quan sát rõ biến động và xu hướng tăng của nguồn thu nhập của các shop.

In [ ]:
import plotly.graph_objs as go
    # Tạo đối tượng Figure và Axes
plt.rcParams['figure.dpi'] = 360


df_income['month']=df_income['month'].astype(str)
df_income['year']=df_income['year'].astype(str)

df_income['month_year']=df_income['month']+'-'+df_income['year']

length=len(df_income['month_year'])
color_markers=[]
for i in range(length):
    color_markers.append('#FF0499')

# Tạo đối tượng đồ thị tương tác
fig = go.Figure()

# Thêm dữ liệu vào đồ thị
fig.add_trace(go.Scatter(x=df_income['month_year'], y=df_income['income'], mode='markers+lines', name='line',
                            marker=dict(size=10, color=color_markers)))

# Cấu hình đồ thị
fig.update_layout(title='Biểu đồ biểu diễn tổng thu nhập của tất cả shop qua từng tháng',
                yaxis_title='Số tiền',
                xaxis_title='Tháng-Năm')

# Hiển thị biểu đồ
fig.show()

- Nhận xét: 
    + Tổng thu nhập của các shop sẽ có xu hướng giảm ở giữa năm, từ tháng 4 đến tháng 7. Vì thời điểm này thường là mùa xuân và mùa hè đầu, nhưng vẫn còn khá lạnh và mưa ở nhiều khu vực, khiến người dân không có nhu cầu mua sắm đồ dùng cho mùa hè. Bên cạnh đó, trong thời gian này, không có các ngày lễ lớn hoặc sự kiện quan trọng nào.
    + Tổng thu nhập các shop tăng cao vào cuối năm. Có thể thấy, trong biểu đồ, đường biểu diễn giá trị đạt đỉnh ở các tháng 12. Lý giải là vì, Ở Nga, tháng 12 được coi là một tháng đặc biệt vì nó gần Giáng sinh và Lễ Tạ ơn Chúa. Do đó, vào tháng 12, nhu cầu mua sắm tăng cao khi mọi người chuẩn bị cho các ngày lễ này. Ngoài ra, tháng 12 cũng là tháng cuối cùng của năm, và nhiều người sẽ dành thời gian để mua sắm đồ để tặng người thân và bạn bè trong dịp kỷ niệm năm mới.
    > Chọn tháng 12 để phân tích tương quan các trường dữ liệu.

#### Bước 3: Lọc những hoạt động mua bán vào tháng 12 và tiến hành gom nhóm theo `shop_id` để tính số lượng mặt hàng đã bán của mỗi shop.

In [ ]:
df_temp=sales_train_df.copy()
df_temp=df_temp.loc[df_temp['date'].dt.month==12]
grouped_shop=df_temp.groupby('shop_id')
grouped_shop=grouped_shop.count().reset_index()[['shop_id','item_id']].sort_values('item_id',ascending=False)
grouped_shop.head()

#### Bước 4: Trực quan bằng `Tree map`.

- `Tree map` rất phù hợp để trực quan cho dữ liệu có nhiều loại phân biệt khác nhau.
> Dùng `tree map` để có thể quan sát rõ hơn nhiều shop và số lượng mặt hàng của mỗi shop bán được.

In [ ]:
plt.rcParams['figure.dpi'] = 360

import plotly.express as px

# Tạo Treemap
fig = px.treemap(grouped_shop, path=['shop_id'], values='item_id', color='shop_id',
                 hover_data=['shop_id'], color_discrete_sequence=px.colors.qualitative.D3_r,
                 width=900, height=600, title='Biểu đồ biểu diễn số lượng mặt hàng của các shop vào tháng 12')

# Cấu hình thông tin hiển thị
fig.update_traces(textinfo='label+value', hovertemplate='<b>%{label}</b><br>Value: %{value}')


# Hiển thị biểu đồ
fig.show()

#### Bước 5: Tiến hành gom nhóm `shop_id` để tính tổng thu nhập mỗi shop vào tháng 12.

In [ ]:
grouped_shop_income=df_temp.groupby('shop_id').sum().reset_index()[['shop_id','item_price']]
grouped_shop_income['count_id']=grouped_shop['item_id']
grouped_shop_income.head()

#### Bước 6: Trực quan bằng `bubble chart`.
- `Bubble chart` rất phù hợp để xét độ lớn giữa các dữ liệu và đưa dữ liệu các dữ liệu có giá trị quy định lớn nổi lên.
> Dùng `bubble chart` để quan sát các shop nào có thu nhập cao để tương quan với `tree map` biểu diễn số lượng mặt hàng của các shop.

In [ ]:
import plotly.express as px

# Tạo Bubble chart
fig = px.scatter(grouped_shop_income, x='shop_id', y='count_id', size='item_price', color='shop_id',
                 hover_data=['shop_id'], text='shop_id')

# Cấu hình tên và tiêu đề trục
fig.update_xaxes(title_text='Shop ID')
fig.update_yaxes(title_text='Số lượng sản phẩm')

# Hiển thị biểu đồ
fig.show()
    


- Nhận xét: 
    - Các shop có thu nhập trên 10,000 chỉ tính ở tháng 12 có id là: 31, 25, 54, 57, 28, 27, 42 (theo thứ tự giảm dần).
    - Hầu hết các shop này đều có trên 10,000 mặt hàng khác nhau được bán.

> Tương quan 2 biểu đồ, ta thấy các shop có số lượng mặt hàng cao nhất cũng theo thứ tự giảm dần đều tương ứng các shop có số tiền thu nhập cao nhất vào tháng 12. 
    > Ta có thể thấy, shop càng có đa dạng số lượng mặt hàng bán được thì sẽ có số tiền bán được càng cao.

### Tìm ra loại sản phẩm trending trong các khoảng thời gian của năm. Tương quan về số lượng các mặt hàng được bán và giá trị của chúng trong nhóm trending.

#### Bước 1: Tiến hành gom nhóm và xác định loại sản phẩm có lượng bán chạy cao nhất.

In [ ]:
df_trending=sales_train_df.copy()
df_category = items_df.copy()
df_item_category = item_category_df.copy()
# df_item_category.head()

df_trending = df_trending.merge(df_category[['item_id', 'item_category_id']], on='item_id', how='left')
df_trending = df_trending.merge(df_item_category[['item_category_id', 'item_category_name']], on='item_category_id', how='left')



df_trending['month']=df_trending['date'].dt.month
df_trending['year']=df_trending['date'].dt.year.astype(str)
df_trending = df_trending.groupby(['year', 'month', 'item_category_id', "item_category_name"],)['item_cnt_day'].sum().reset_index()
# df_trending.head()

df_trending = df_trending.loc[df_trending.groupby(['year', 'month'])['item_cnt_day'].idxmax()][['year', 'month', 'item_category_id',"item_category_name", 'item_cnt_day']].reset_index(drop=True)
df_trending.head()

In [ ]:
import plotly.graph_objs as go
    # Tạo đối tượng Figure và Axes
plt.rcParams['figure.dpi'] = 360


df_trending['month']=df_trending['month'].astype(str)
df_trending['year']=df_trending['year'].astype(str)

df_trending['month_year']=df_trending['month']+'-'+df_trending['year']

length=len(df_trending['month_year'])


# Tạo đối tượng đồ thị tương tác
fig = go.Figure()

# Thêm dữ liệu vào đồ thị

fig.add_trace(go.Bar(x=df_trending['month_year'], y=df_trending['item_category_id'], name='bar'))

# Cấu hình đồ thị
fig.update_layout(title='Biểu đồ biểu diễn loại sản phẩm có nhiều lượt mua nhất theo tháng',
                yaxis_title='ID sản phẩm',
                xaxis_title='Tháng-Năm')

# Hiển thị biểu đồ
fig.show()

- Nhận xét: 
    + Ta thấy loại sản phẩm bán chạy nhất trong khoảng thời gian được cung cấp trong dữ liệu là loại 40 (Кино - DVD).
    + Mặc dù có một số loại khác: 30, 20 nhưng số lượng không đáng kể.
    > Chọn loại sản phẩm 40 để phân tích sản phẩm nào trong nhóm 40 là bán chạy nhất.

In [ ]:
df_trending=sales_train_df.copy()
df_category = items_df.copy()
# df_item_category.head()

df_trending = df_trending.merge(df_category[['item_id', 'item_category_id']], on='item_id', how='left')



df_trending['month']=df_trending['date'].dt.month
df_trending['year']=df_trending['date'].dt.year.astype(str)
df_trending=df_trending[df_trending['item_category_id'] == 40]

df_max_item_cnt_day = df_trending.loc[df_trending.groupby(['year', 'month'])['item_cnt_day'].idxmax()][["year","month","item_id","item_cnt_day"]].reset_index(drop=True)
df_max_item_cnt_day['month']=df_max_item_cnt_day['month'].astype(str)
df_max_item_cnt_day['year']=df_max_item_cnt_day['year'].astype(str)

df_max_item_cnt_day['month_year']=df_max_item_cnt_day['month']+'-'+df_max_item_cnt_day['year']

df_max_item_cnt_day.head()

In [ ]:
import plotly.express as px

# Tạo Bubble chart
fig = px.scatter(df_max_item_cnt_day, x='month_year', y='item_cnt_day', size='item_id', color='item_id',
                 hover_data=['item_id'], text='item_id')

# Cấu hình tên và tiêu đề trục
fig.update_xaxes(title_text='Month - Year')
fig.update_yaxes(title_text='item_cnt_day')

# Hiển thị biểu đồ
fig.show()
    


- Nhận xét: 
    - Nhìn chung, số lượng hàng bán được cao nhất trong mỗi tháng có xu hướng giảm từ năm 2013 đến năm 2015. Đặc biệt là mốc thời gian tháng 4 2014, kể từ sau thời điểm này, số lượng hàng bán được cao nhất chỉ loang quanh ở mức 10.
    - Ta thấy có 3 phân khúc về số lượng hàng được bán: từ 10 trở xuống, 10 đến 30 và trên 30. Trong đó, phân khúc từ 10 trở xuống lớn hơn cả, và có xu hướng xuất hiện ngày càng nhiều ở khoảng thời gian cuối của khảo sát. Điều này cho thấy các mặt hàng dần đạt đến ngưỡng bão hòa khi không có một hàng nào có sự nổi bật về doanh số.

In [ ]:
df_trending=sales_train_df.copy()
df_category = items_df.copy()
# df_item_category.head()

df_trending = df_trending.merge(df_category[['item_id', 'item_category_id']], on='item_id', how='left')



df_trending['month']=df_trending['date'].dt.month
df_trending['year']=df_trending['date'].dt.year.astype(str)
df_trending=df_trending[df_trending['item_category_id'] == 40]
df_trending['income']=df_trending['item_cnt_day']*df_trending['item_price']
df_max_item_cnt_day_income = df_trending.loc[df_trending.groupby(['year', 'month'])['item_cnt_day'].idxmax()][["year","month","item_id","income"]].reset_index(drop=True)

df_max_item_cnt_day_income['month']=df_max_item_cnt_day_income['month'].astype(str)
df_max_item_cnt_day_income['year']=df_max_item_cnt_day_income['year'].astype(str)

df_max_item_cnt_day_income['month_year']=df_max_item_cnt_day_income['month']+'-'+df_max_item_cnt_day_income['year']

df_max_item_cnt_day_income.head()

In [ ]:
import plotly.express as px

# Tạo Bubble chart
fig = px.scatter(df_max_item_cnt_day_income, x='month_year', y='income', size='item_id', color='item_id',
                 hover_data=['item_id'], text='item_id')

# Cấu hình tên và tiêu đề trục
fig.update_xaxes(title_text='Month - Year')
fig.update_yaxes(title_text='item_cnt_day')

# Hiển thị biểu đồ
fig.show()
    


- Nhận xét: 
    - Nhìn chung, giá trị hàng bán được cao nhất trong mỗi tháng có xu hướng giảm từ năm 2013 đến năm 2015. Đặc biệt là mốc thời gian tháng 4 2014, kể từ sau thời điểm này, giá trị hàng bán được cao nhất chỉ loang quanh ở mức 5000.
    - Ta thấy có 3 phân khúc về số lượng hàng được bán: từ 5000 trở xuống, 5000 đến 10000 và trên 10000. Trong đó, phân khúc từ 5000 trở xuống lớn hơn cả, và có xu hướng xuất hiện ngày càng nhiều ở khoảng thời gian cuối của khảo sát. Điều này cho thấy các mặt hàng dần đạt đến ngưỡng bão hòa.

> Từ 3 biểu đồ ta có thể thấy, nhóm sản phẩm bán được nhất là nhóm 40, tuy nhiên, trong từng giai đoạn, số lượng sản phẩm bán ra ngày một bão hòa, kèm theo đó là tổng tiền thu được ngày càng giảm khi số sản phẩm bán được giảm theo. 